# 1 h fix bug: Vì cứ để code cũ trong script.js thì chạy 
mà cứ cho script.js sử dụng file config.js thì truyền vào HTML lại ko work

=> Test 
```
viết 1 file script.js đơn giản để lấy thông tin từ file config.js và hiển thị trên .html
```
```bash
// Xuất một object config chứa các cấu hình API URL cho từng môi trường
export const config = {
    // Môi trường development (phát triển): 
    // Sử dụng localhost vì frontend và backend chạy trên cùng máy tính
    development: {
      apiUrl: 'http://localhost:3000/api'
    },

    // Môi trường docker internal:
    // Sử dụng tên service 'backend' được định nghĩa trong docker-compose
    // để giao tiếp giữa các container trong mạng nội bộ Docker
    dockerInternal: {
      apiUrl: 'http://backend:3000/api'  // Docker service name
    },

    // Môi trường production (triển khai):
    // Sử dụng IP public của server để frontend có thể gọi API
    // từ bất kỳ đâu trên internet
    production: {
      apiUrl: 'http://103.253.20.13:3000/api' // IP public
    }
};

// Lấy giá trị môi trường từ biến môi trường NODE_ENV
// Nếu không có giá trị, mặc định sử dụng 'development'
const environment = process.env.NODE_ENV || 'development';

// Xuất cấu hình tương ứng với môi trường hiện tại
// Ví dụ: Nếu NODE_ENV='production' thì sẽ xuất config.production
export default config[environment];

```

# 2. Đã tìm được bug 

Về vị trí thì như này là oke
```html
    <!-- Đường dẫn tới thư mục js -->
    <script type="module" src="js/script.js"></script>
```

```js
// script.js
// Import từ config.js trong cùng thư mục js
import config from './config.js';
```


LỖI Ở FILE config.js

Như này 
```js
const config = {
    development: {
        apiUrl: 'http://localhost:3000/api'
    },
    dockerinternal: {
        apiUrl: 'http://backend:3000/api'
    },
    production: {
        apiUrl: 'http://103.253.20.13:3000/api'
    }
};

// Lấy giá trị môi trường từ biến môi trường NODE_ENV
// Nếu không có giá trị, mặc định sử dụng 'development'
const environment = process.env.NODE_ENV || 'development';

// Xuất cấu hình tương ứng với môi trường hiện tại
// Ví dụ: Nếu NODE_ENV='production' thì sẽ xuất config.production
export default config[environment];

```

```js  
// config.js
const config = {
  development: {
      apiUrl: 'http://localhost:3000/api'
  },
  dockerInternal: {
      apiUrl: 'http://backend:3000/api'
  },
  production: {
      apiUrl: 'http://103.253.20.13:3000/api'
  }
};

export default config;

```



Ah, tôi hiểu vấn đề rồi! Có sự khác biệt giữa hai cách export config:

1. **File config.js ban đầu của bạn:**
```javascript
// Export một giá trị duy nhất dựa trên environment
export default config[environment];  // Xuất URL của một môi trường cụ thể
```

2. **File config.js đang hoạt động:**
```javascript
// Export toàn bộ object config
export default config;  // Xuất toàn bộ object chứa tất cả môi trường
```

Lý do bug:
1. Trong phiên bản đầu tiên, bạn export một giá trị string (URL) duy nhất dựa trên `process.env.NODE_ENV`
2. `process.env.NODE_ENV` không tồn tại trong môi trường browser
3. Khi script.js cố gắng truy cập `config[environment]`, nó không thể vì config là một string, không phải object

Để sửa, bạn có 2 lựa chọn:

1. **Cách 1: Giữ logic chọn environment trong config.js**
```javascript
const config = {
    development: {
        apiUrl: 'http://localhost:3000/api'
    },
    dockerinternal: {
        apiUrl: 'http://backend:3000/api'
    },
    production: {
        apiUrl: 'http://103.253.20.13:3000/api'
    }
};

// Export cả object config và environment mặc định
export const defaultEnvironment = 'development';
export default config;
```

Và trong script.js:
```javascript
import config, { defaultEnvironment } from './config.js';

document.addEventListener('DOMContentLoaded', () => {
    // ... code khác ...
    updateApiUrl(defaultEnvironment); // Sử dụng environment mặc định
});
```

2. **Cách 2: Chuyển logic chọn environment sang script.js**
```javascript
// config.js
const config = {
    development: {
        apiUrl: 'http://localhost:3000/api'
    },
    dockerinternal: {
        apiUrl: 'http://backend:3000/api'
    },
    production: {
        apiUrl: 'http://103.253.20.13:3000/api'
    }
};

export default config;
```

```javascript
// script.js
import config from './config.js';

// Chuyển logic chọn environment vào đây
const defaultEnvironment = 'development';

document.addEventListener('DOMContentLoaded', () => {
    // ... code khác ...
    updateApiUrl(defaultEnvironment);
});
```

Tôi khuyên bạn nên sử dụng **Cách 2** vì:
1. Đơn giản và dễ hiểu hơn
2. Tách biệt rõ ràng: config.js chỉ chứa dữ liệu, script.js xử lý logic
3. Tránh sử dụng `process.env` trong browser environment

Nếu bạn thực sự cần sử dụng environment variables trong production, bạn nên:
1. Sử dụng một build tool (như Webpack, Vite)
2. Hoặc sử dụng biến môi trường từ server của bạn

Bạn muốn sử dụng cách nào?